Задачи:

Заменить категориальные признаки на числовые 

Построить линейную регрессию для прогнозирования
стоимости квартир с сайта avito и оценить точность.

Масштабировать признаки и оценить их важность в
линейной модели.

Построить полиномиальную регрессию и оценить точность.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Perm_IT_Univer/ML/4_Avito_regression/avito2.csv')
df

,address,floor,plan,price,square,number of storeys
0,р-н Дзержинский,14,2-к квартира,4100000.0,59.0,25
1,р-н Дзержинский,5,2-к квартира,3900000.0,52.2,10
2,р-н Индустриальный,18,2-к квартира,3873200.0,57.6,19
3,р-н Дзержинский,2,4-к квартира,2890000.0,61.0,5
4,р-н Ленинский,13,Своб. планировка,8554000.0,132.0,25
...,...,...,...,...,...,...
2446,р-н Дзержинский,22,3-к квартира,5716000.0,82.2,25
2447,р-н Мотовилихинский,2,3-к квартира,3350000.0,56.7,5
2448,р-н Мотовилихинский,4,2-к квартира,2370000.0,43.0,5
2449,р-н Орджоникидзевский,2,2-к квартира,1450000.0,44.9,3


In [3]:
X = df
y = df['price']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=142, shuffle=True)

In [5]:
def feature_generation(X, feature1, feature2):
  weight = X.groupby(feature1).agg({feature2:'mean'}) \
    .rename(columns={feature2:'weight_of_'+str(feature1)}) \
    .sort_values('weight_of_'+str(feature1), ascending=False).reset_index()
  return weight.round(1)

In [6]:
# добавляем новые признак - средняя стоимость квартир в районе
weight_of_address = feature_generation(X_train, 'address','price')
X_train = X_train.merge(weight_of_address, on='address', how='left')
X_test = X_test.merge(weight_of_address, on='address', how='left')
X_train.head()

,address,floor,plan,price,square,number of storeys,weight_of_address
0,р-н Кировский,4,1-к квартира,1000000.0,30.7,5,2379450.1
1,р-н Мотовилихинский,3,3-к квартира,3555000.0,70.0,5,3153071.8
2,р-н Мотовилихинский,6,3-к квартира,7500000.0,92.3,14,3153071.8
3,р-н Свердловский,5,2-к квартира,1750000.0,45.0,5,3648945.3
4,р-н Дзержинский,3,1-к квартира,2200000.0,31.0,5,3554370.6


In [7]:
# добавляем новые признак - средняя стоимость квартиры с данной планировкой
weight_of_plan = feature_generation(X_train, 'plan','price')
X_train = X_train.merge(weight_of_plan, on='plan', how='left')
X_test = X_test.merge(weight_of_plan, on='plan', how='left')
X_train.head()

,address,floor,plan,price,square,number of storeys,weight_of_address,weight_of_plan
0,р-н Кировский,4,1-к квартира,1000000.0,30.7,5,2379450.1,2280665.0
1,р-н Мотовилихинский,3,3-к квартира,3555000.0,70.0,5,3153071.8,4370771.5
2,р-н Мотовилихинский,6,3-к квартира,7500000.0,92.3,14,3153071.8,4370771.5
3,р-н Свердловский,5,2-к квартира,1750000.0,45.0,5,3648945.3,3109588.0
4,р-н Дзержинский,3,1-к квартира,2200000.0,31.0,5,3554370.6,2280665.0


In [8]:
#полный набор числовых признаков в датасэте
X_train_full = X_train.drop(columns=['address','plan', 'price'])
X_test_full = X_test.drop(columns=['address','plan', 'price'])
X_test_full.head()

,floor,square,number of storeys,weight_of_address,weight_of_plan
0,5,52.0,10,3294439.3,3109588.0
1,6,45.0,9,3153071.8,3109588.0
2,8,71.3,10,2134392.2,4370771.5
3,8,52.2,17,3554370.6,2280665.0
4,3,47.7,25,3648945.3,3109588.0


In [9]:
scaler = StandardScaler()
X_train_full = pd.DataFrame(scaler.fit_transform(X_train_full), columns=X_train_full.columns)
X_test_full = pd.DataFrame(scaler.fit_transform(X_test_full), columns=X_test_full.columns)
X_test_full

,floor,square,number of storeys,weight_of_address,weight_of_plan
0,-0.178822,-0.005725,-0.183966,-0.120651,-0.121191
1,0.023038,-0.277051,-0.328777,-0.259815,-0.121191
2,0.426759,0.742362,-0.183966,-1.262612,0.866418
3,0.426759,0.002028,0.829714,0.135227,-0.770305
4,-0.582543,-0.172397,1.988205,0.228328,-0.121191
...,...,...,...,...,...
731,0.628619,-0.994129,1.988205,-1.262612,-0.869186
732,-0.178822,-0.362326,-0.908022,-0.120651,0.866418
733,1.032340,-0.587139,0.395280,-0.259815,-0.770305
734,-0.178822,-1.129793,-0.328777,0.228328,-0.869186


In [10]:
#числовые признаки + weight_of_address
X_train_address = X_train_full.drop(columns=['weight_of_plan'])
X_test_address = X_test_full.drop(columns=['weight_of_plan'])
X_train_address.head()

,floor,square,number of storeys,weight_of_address
0,-0.390616,-0.860732,-0.857955,-1.050485
1,-0.589638,0.642715,-0.857955,-0.235689
2,0.007427,1.495816,0.521476,-0.235689
3,-0.191595,-0.313676,-0.857955,0.286576
4,-0.589638,-0.849255,-0.857955,0.186968


In [11]:
#числовые признаки + weight_of_plan
X_train_plan = X_train_full.drop(columns=['weight_of_address'])
X_test_plan = X_test_full.drop(columns=['weight_of_address'])
X_train_plan.head()

,floor,square,number of storeys,weight_of_plan
0,-0.390616,-0.860732,-0.857955,-0.660864
1,-0.589638,0.642715,-0.857955,0.599211
2,0.007427,1.495816,0.521476,0.599211
3,-0.191595,-0.313676,-0.857955,-0.161126
4,-0.589638,-0.849255,-0.857955,-0.660864


In [12]:
#только числовые признаки
X_train = X_train_full.drop(columns=['weight_of_address', 'weight_of_plan'])
X_test = X_test_full.drop(columns=['weight_of_address', 'weight_of_plan'])
X_train.head()

,floor,square,number of storeys
0,-0.390616,-0.860732,-0.857955
1,-0.589638,0.642715,-0.857955
2,0.007427,1.495816,0.521476
3,-0.191595,-0.313676,-0.857955
4,-0.589638,-0.849255,-0.857955


In [13]:
reg = linear_model.LinearRegression()
r2 = pd.DataFrame(columns=['train','test'])

In [35]:
reg.fit(X_train_full, y_train)
train_r2 = r2_score(y_train, reg.predict(X_train_full))
test_r2 = r2_score(y_test, reg.predict(X_test_full))
print('train_r2=',train_r2,'test_r2=',test_r2)
print(pd.DataFrame({'feature':X_train_full.columns,'coef':reg.coef_.round(0)}).sort_values(by='coef', ascending=False))

train_r2= 0.6676293133344682 test_r2= 0.6084185684709369
             feature       coef
1             square  1925190.0
3  weight_of_address   449134.0
2  number of storeys   238703.0
0              floor   157492.0
4     weight_of_plan   120650.0


In [41]:
#полиномиальная регрессия
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)

X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.fit_transform(X_test)

X_poly_train_address = poly.fit_transform(X_train_address)
X_poly_test_address = poly.fit_transform(X_test_address)

X_poly_train_plan = poly.fit_transform(X_train_plan)
X_poly_test_plan = poly.fit_transform(X_test_plan)

X_poly_train_full = poly.fit_transform(X_train_full)
X_poly_test_full = poly.fit_transform(X_test_full)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.0,-0.178822,-0.005725,-0.183966,-0.120651,-0.121191,0.031977,0.001024,0.032897,0.021575,0.021672,0.000033,0.001053,0.000691,0.000694,0.033843,0.022196,0.022295,0.014557,0.014622,0.014687
1,1.0,0.023038,-0.277051,-0.328777,-0.259815,-0.121191,0.000531,-0.006383,-0.007574,-0.005986,-0.002792,0.076757,0.091088,0.071982,0.033576,0.108094,0.085421,0.039845,0.067504,0.031487,0.014687
2,1.0,0.426759,0.742362,-0.183966,-1.262612,0.866418,0.182123,0.316810,-0.078509,-0.538831,0.369752,0.551101,-0.136569,-0.937315,0.643196,0.033843,0.232277,-0.159391,1.594189,-1.093950,0.750680
3,1.0,0.426759,0.002028,0.829714,0.135227,-0.770305,0.182123,0.000865,0.354088,0.057710,-0.328735,0.000004,0.001682,0.000274,-0.001562,0.688426,0.112200,-0.639133,0.018286,-0.104166,0.593370
4,1.0,-0.582543,-0.172397,1.988205,0.228328,-0.121191,0.339356,0.100428,-1.158214,-0.133011,0.070599,0.029721,-0.342760,-0.039363,0.020893,3.952960,0.453962,-0.240953,0.052133,-0.027671,0.014687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,1.0,0.628619,-0.994129,1.988205,-1.262612,-0.869186,0.395162,-0.624929,1.249824,-0.793702,-0.546387,0.988293,-1.976533,1.255200,0.864083,3.952960,-2.510332,-1.728120,1.594189,1.097445,0.755484
732,1.0,-0.178822,-0.362326,-0.908022,-0.120651,0.866418,0.031977,0.064792,0.162374,0.021575,-0.154935,0.131280,0.329000,0.043715,-0.313925,0.824505,0.109554,-0.786727,0.014557,-0.104534,0.750680
733,1.0,1.032340,-0.587139,0.395280,-0.259815,-0.770305,1.065726,-0.606127,0.408063,-0.268217,-0.795217,0.344732,-0.232084,0.152547,0.452276,0.156246,-0.102700,-0.304486,0.067504,0.200137,0.593370
734,1.0,-0.178822,-1.129793,-0.328777,0.228328,-0.869186,0.031977,0.202032,0.058793,-0.040830,0.155430,1.276432,0.371450,-0.257963,0.982000,0.108094,-0.075069,0.285768,0.052133,-0.198459,0.755484


In [37]:
reg = linear_model.LinearRegression()
#r2_poly = pd.DataFrame(columns=['train','test'])

In [42]:
reg.fit(X_poly_train_full, y_train)
train_r2 = r2_score(y_train, reg.predict(X_poly_train_full))
test_r2 = r2_score(y_test, reg.predict(X_poly_test_full))
print('train_r2=',train_r2,'test_r2=',test_r2)

train_r2= 0.836382474480466 test_r2= 0.8791555413751134
